In [ ]:
!pip install --upgrade typing_extensions

In [ ]:
!pip install openai==0.28

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import openai
import os

In [2]:
openai.api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# Different functions
def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png")])
    if file_path:
        image_path.set(file_path)

def select_audio():
    file_path = filedialog.askopenfilename(filetypes=[("Audio files", "*.mp3")])
    if file_path:
        audio_path.set(file_path)

def summarize():
    img = image_path.get()
    aud = audio_path.get()
    txt = text_input.get("1.0", tk.END).strip()

    if not img and not aud and not txt:
        messagebox.showerror("Input Error", "Please provide at least one input (image, audio, or text).")
        return

    prompt = f"Image: {img}\nAudio: {aud}\nText: {txt}\nplease conclude the customer's\
               requirements from the information provided.Answer as short as possible"
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    summary_output.set(response['choices'][0]['message']['content'].strip())

In [3]:
# Create the main window
root = tk.Tk()
root.title("Summarizer")

# Define variables
image_path = tk.StringVar()
audio_path = tk.StringVar()
summary_output = tk.StringVar()

# Create and layout widgets
tk.Label(root, text="Select Image:").grid(row=0, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=image_path, width=50).grid(row=0, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=select_image).grid(row=0, column=2, padx=10, pady=10)

tk.Label(root, text="Select Audio:").grid(row=1, column=0, padx=10, pady=10)
tk.Entry(root, textvariable=audio_path, width=50).grid(row=1, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=select_audio).grid(row=1, column=2, padx=10, pady=10)

tk.Label(root, text="Enter Text:").grid(row=2, column=0, padx=10, pady=10)
text_input = tk.Text(root, height=10, width=50)
text_input.grid(row=2, column=1, columnspan=2, padx=10, pady=10)

tk.Button(root, text="Summary", command=summarize).grid(row=3, column=1, padx=10, pady=20)

tk.Label(root, text="Summary Output:").grid(row=4, column=0, padx=10, pady=10)
tk.Label(root, textvariable=summary_output, wraplength=400, justify="left").grid(row=4, column=1, columnspan=2, padx=10, pady=10)

In [4]:
# The main loop
root.mainloop()